In [12]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import os
import folium
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_pandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

BUFFER_IN_METERS = 10 * 1000 # 10km
SNOW_EX_GEOJSON_FILENAME = "snow_ex_all_stations.geojson"
REFRESH_SNOW_EX_GEOJSON = False

if REFRESH_SNOW_EX_GEOJSON or not os.path.isfile(SNOW_EX_GEOJSON_FILENAME):
    # Get the snow ex sites
    SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

    engine, session = get_db(SNOWEX_DB_NAME)
    qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.latitude, SiteData.longitude)

    df_snow_ex = query_to_pandas(qry, engine).drop_duplicates(subset=["site_id"])
    gdf_snow_ex = gpd.GeoDataFrame(
        df_snow_ex, 
        geometry=gpd.points_from_xy(df_snow_ex.longitude, df_snow_ex.latitude),
        crs=4326
    )
    gdf_snow_ex.to_file(SNOW_EX_GEOJSON_FILENAME, driver='GeoJSON')
    #gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)
else:
    gdf_snow_ex = gpd.read_file(SNOW_EX_GEOJSON_FILENAME)
    
print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snow Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex_buffer.to_crs(4326).explore()
snow_ex_map = gdf_snow_ex.to_crs(4326).explore(m=snow_ex_map)


# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
msnt_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=MSNT&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_msnt = gpd.read_file(msnt_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_sntl = pd.concat([gdf_msnt, gdf_sntl])
gdf_sntl = gdf_sntl[gdf_sntl["beginDate"].dt.year <= 2017]
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
).set_geometry("geometry")
gdf_sntl_clipped = gdf_sntl.clip(gdf_snow_ex_buffer)
sntl_map = gdf_sntl_clipped.explore()
snow_ex_map = gdf_sntl_clipped.explore(m=snow_ex_map)
print(f"{len(gdf_sntl_clipped)} AWDB sites were found within {BUFFER_IN_METERS / 1000:.0f} km of SnowEx Sites")
_ = [print(f'* {row["site_name"]} ({row["site_id"]})') for idx, row in gdf_sntl_clipped.sort_values("site_name").iterrows()]

The CRS of the Snow Ex metadata is - epsg:4326
The bounding box of the snow Ex sites is - [-120.29898   35.85793 -105.54616   44.30447]
The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:4326, all good!
28 AWDB sites were found within 10 km of SnowEx Sites
* Banner Summit (312:ID:SNTL)
* Bogus Basin (978:ID:SNTL)
* Brighton (366:UT:SNTL)
* Butte (380:CO:SNTL)
* Caples Lake (CAP:CA:MSNT)
* Carson Pass (1067:CA:SNTL)
* Devils Postpile (DPO:CA:MSNT)
* Fool Creek (1186:CO:SNTL)
* Forestdale Creek (1049:CA:SNTL)
* Idarado (538:CO:SNTL)
* Independence Camp (539:CA:SNTL)
* Independence Creek (540:CA:SNTL)
* Independence Lake (541:CA:SNTL)
* Joe Wright (551:CO:SNTL)
* Mammoth Pass (MHP:CA:MSNT)
* Mesa Lakes (622:CO:SNTL)
* Mill-D North (628:UT:SNTL)
* Mineral Creek (629:CO:SNTL)
* Mores Creek Summit (637:ID:SNTL)
* Niwot (663:CO:SNTL)
* Red Mountain Pass (713:CO:SNTL)
* Sawtooth (1251:CO:SNTL)
* Schneiders (SCN:CA:MSNT)
* Schofield Pass

In [ ]:
gdf.to_file('snow_ex_all_stations.geojson', driver='GeoJSON')

In [ ]:
snow_ex_map

In [ ]:
sntl_map

In [ ]:
try:
    import pygeos
except ImportError:
    print("Need to install pyGEOS - pip install pygeos")
    gdf_nearest = gpd.GeoDataFrame()
else:
    gdf_nearest = gpd.sjoin_nearest(gdf_snow_ex.to_crs(26912), gdf_sntl.to_crs(26912), distance_col="distance").set_geometry(col="geometry").to_crs(4326)
gdf_nearest

In [ ]:
if not gdf_nearest.empty:
    nearest_map = gdf_nearest.explore(
        "distance", 
        tooltip=[i for i in gdf_nearest.columns if not i.lower().startswith("geo")],
        marker_kwds={"radius": 10},
        vmin=0,
    )
else:
    nearest_map = "Need to install pyGEOS - pip install pygeos"
nearest_map

In [ ]:

import os
from os import path, makedirs

curr_path = os.getcwd()
output_dir = path.join(curr_path, "output")
makedirs(output_dir, exist_ok=True)
shp_dir = path.join(output_dir, "shp")
makedirs(shp_dir, exist_ok=True)
geojson_dir = path.join(output_dir, "geojson")
makedirs(geojson_dir, exist_ok=True)

snow_ex_dir = path.join(shp_dir, "snow-ex")
makedirs(snow_ex_dir, exist_ok=True)
gdf_snow_ex.to_crs(4326, inplace=True)
gdf_snow_ex.set_geometry(col="geometry", inplace=True)
gdf_snow_ex.to_file(path.join(snow_ex_dir, "snow_ex_stations.shp"), crs=4326)
gdf_snow_ex.to_file(path.join(geojson_dir, "snow_ex_stations.json"), crs=4326, driver='GeoJSON')
                    
sntl_dir = path.join(shp_dir, "sntl")
makedirs(sntl_dir, exist_ok=True)
gdf_sntl.to_crs(4326, inplace=True)
gdf_sntl.to_file(path.join(sntl_dir, "sntl_stations.shp"), crs=4326)
gdf_sntl.to_file(path.join(geojson_dir, "sntl_stations.json"), crs=4326, driver='GeoJSON')

try:
    import pygeos
except ImportError:
    print("Need to install pyGEOS - pip install pygeos")
else:
    nearest_dir = path.join(shp_dir, "nearest")
    makedirs(nearest_dir, exist_ok=True)
    gdf_nearest.to_file(path.join(nearest_dir,  "nearest_stations.shp"), crs=4326)
    gdf_nearest.to_file(path.join(geojson_dir,  "nearest_stations.json"), crs=4326, driver='GeoJSON')